# Convolutional Neural Network For Crop Prediction and Forecasting

This script is for Group 12, Machine Learning Covolutional Neural Network by Gil and Marcus. 

# Libraries

This first section of the script is used to import the various libraries that are required for the model and the statistics we need to be outputted at the end. 

In [ ]:
import os
import datetime
import csv
from google.colab import drive
drive.mount('/content/drive')

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import h5py
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

Mounted at /content/drive


# Constants and Global Variables

This section is dedicated to determining the global variables that need to be carried over from the various modular components within the script. The Constants are the constants that we found worked best for creating the windows and training the models.


In [ ]:
# CONSTANTS

OUT_STEPS = 19
MAX_EPOCHS = 50

# GLOBAL VARIABLES

multi_val_performance = {}
multi_performance = {}
train_df = None
val_df = None
test_df = None
num_features = []
column_indices = []
crop_type = []
caruid = []
history = []

# Extract Usable Models

This section is for extracting the various models that would work with the CNN. Through implementation we discovered that the CNN does not accept CSV files that contain less than around 350 rows. To remedy this problem, we queried all the available CSV files and put the names into their own file. This section queries that CSV file to retrieve all models that fulfill the 350 row requirement.

In [ ]:
usable_models = []
usable_path = "/content/drive/MyDrive/GEOG481/compiled/{0}usable.csv".format("Sprwht") # Replace Sprwhat with either Barley or Canola
with open(usable_path, newline = '') as csvfile: 
  csvreader = csv.reader(csvfile, delimiter = ' ', quotechar = '|')
  for rows in csvreader:
    usable_models.append(rows[0].split(",")[0].split(".")[0])
usable_models.pop(0) # Remove "File Name" from the rows

'File Name'

# Data Shaping and Manipulation

The first section of the script involves taking the data from the CSV files and manipulating it to better fit into our CNN model. The first function, *data manipulation* is the manipulation of data that goes into our model. The second function, *create_dataset_X* is for configuring our data to be accepted for predictions.

In [ ]:
# Function to split our data into training, validation and test data
def data_manipulation(csv_file):
  global multi_val_performance, multi_performance, train_df, val_df, test_df
  global num_features, column_indices, crop_type, caruid, history

  # This section assigns the correct CSV path and splits it to determine the current Crop Type and CARUID
  csv_path = "/content/drive/MyDrive/GEOG481/david_input/{0}_output_david/{1}.csv".format(csv_file.split("_")[0].lower().capitalize(), csv_file)
  split_path = csv_path.split("/")
  csv_split = split_path[-1].split("_")
  crop_type, caruid = csv_split[0], csv_split[1].split(".")[0]

  # Remove the unwanted value ("Year")
  df = pd.read_csv(csv_path)
  date_time = df.pop('Year')

  # Plotting Function
  plot_cols = ['Yield']
  plot_features = df[plot_cols]
  plot_features.index = date_time
  _ = plot_features.plot(subplots=True)

  column_indices = {name: i for i, name in enumerate(df.columns)}

  # This part splits the data into different categories (70%, 20%, 10%)
  n = len(df)
  train_df = df[0:int(n*0.7)]
  val_df = df[int(n*0.7):int(n*0.9)]
  test_df = df[int(n*0.9):]

  num_features = df.shape[1]

  # This Section normalizes the data so that it is all scaled properly
  train_mean = train_df.mean()
  train_std = train_df.std()

  train_df = (train_df - train_mean) / train_std
  val_df = (val_df - train_mean) / train_std
  test_df = (test_df - train_mean) / train_std

  # Plotting function
  df_std = (df - train_mean) / train_std
  df_std = df_std.melt(var_name='Column', value_name='Normalized')
  plt.figure(figsize=(12, 6))
  ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
  _ = ax.set_xticklabels(df.keys(), rotation=90)

# Function that creates a dataset for prediction
def create_dataset_X(X, time_steps=1):
    Xs = []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
    return np.array(Xs)

# Creating the Window Generator

This part of the script is used to create the WindowGenerator that acts as the backbone of the model. 

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
      labels = tf.stack(
          [labels[:, :, self.column_indices[name]] for name in self.label_columns],
          axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

def plot(self, model=None, plot_col='Yield', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)
    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [3-Week Average]')

def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)

    ds = ds.map(self.split_window)
    return ds

@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.plot = plot
WindowGenerator.split_window = split_window
WindowGenerator.make_dataset = make_dataset


# Compile and Fit Function

This section is essentially the training procedure bottled into a single function.  

In [ ]:
def compile_and_fit(model, window, patience=5):
  # Checkpoint callback usage
  checkpoint_path = '/content/drive/MyDrive/GEOG481/output/{0}/Checkpoints/{1}/cp.ckpt'.format(crop_type, caruid)
  # checkpoint_path = 'saved_model/Barley_1100/training_1/cp.ckpt'
  
  # Create a callback that saves the model's weights
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=True,
                                                  monitor='val_accuracy',
                                                  verbose=1)
  
  # Changed from meansquarederror() to SparseCategoricalCrossentropy()
  # optimizer changed from tf.optimizer.Adam() to 'adam'
  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer='adam',
                metrics=[tf.metrics.MeanSquaredError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[cp_callback])
  return history

# Model Creation

This is the function that creates the model itself and determines the early statistics for the initial model creation.

In [ ]:
def model_creation(): 
  # Create the window that will be used to help create the model
  multi_window = WindowGenerator(input_width=19,
                                label_width=OUT_STEPS,
                                shift=OUT_STEPS,
                                train_df=train_df,
                                val_df = val_df,
                                test_df = test_df)
  print(multi_window)

  CONV_WIDTH = 3
  multi_conv_model = tf.keras.Sequential([
      # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
      tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
      # Shape => [batch, 1, conv_units]
      tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
      # Shape => [batch, 1, out_steps*features]
      tf.keras.layers.Dense(OUT_STEPS*num_features,
                            kernel_initializer=tf.initializers.zeros()),
      # Shape => [batch, out_steps, features]
      tf.keras.layers.Reshape([OUT_STEPS, num_features])
  ])

  # Train the model once using compile_and_fit
  history = compile_and_fit(multi_conv_model, multi_window)

  IPython.display.clear_output()

  # Check the performance of the new model using the evaluate function
  multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val, verbose=1)
  multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, batch_size=32, verbose=1)
  multi_window.plot(multi_conv_model)

  # Save it in the pre-determined directory
  multi_conv_model.save('/content/drive/MyDrive/GEOG481/output/{0}/Models/{1}/model'.format(crop_type, caruid))

  print("Model Created")

# Model Training

This section is dedicated to performing the model training. We found that the optimal training length for our models were 20 interations using 50 epochs.

In [ ]:
def model_training():
  for i in range(20):
    # Recreate the multi-window, same as model creation
    multi_window = WindowGenerator(input_width=19,
                                  label_width=OUT_STEPS,
                                  shift=OUT_STEPS,
                                  train_df = train_df,
                                  val_df = val_df,
                                  test_df = test_df)

    # Load the model that was saved previously
    new_model = tf.keras.models.load_model('/content/drive/MyDrive/GEOG481/output/{0}/Models/{1}/model'.format(crop_type, caruid))
    # Train the model again
    history = compile_and_fit(new_model, multi_window)
    # Save the model again
    new_model.save('/content/drive/MyDrive/GEOG481/output/{0}/Models/{1}/model'.format(crop_type, caruid))

  # Plot the performance of the new model
  multi_window.plot(new_model)

# Denormalize Predictions

This is a helper function that is used to denormalize the predictions after the data is run through the predict function.

In [ ]:
def denormalize_pred(predictions, train):
  new_values =[]
  train_mean = train_df.mean() # Redetermine the training mean
  train_std = train_df.std() # Redetermine the training std

  for arrays in predictions:
    for predict_vals in arrays:
      denormalized_values = (predict_vals*train_std) + train_mean # Convert the value back to normal
      new_values.append(denormalized_values)

  return new_values

# Prediction

The final technical function in the script is the one that retrieves predictions and denormalizes them for use and comparison. It will use the full dataset as its input and retrieve predictions for comparison.

In [ ]:
def grab_predictions(file_name):
  crop_type = file_name.split("_")[0] 
  caruid = file_name.split("_")[1].split(".")[0]
  
  direct = "/content/drive/MyDrive/GEOG481/david_input/{0}_output_david/".format(crop_type)
  new_model = tf.keras.models.load_model('/content/drive/MyDrive/GEOG481/output/{0}/Models/{1}/model'.format(crop_type, caruid))

  # Retrieve the correct CSV for the model retrieved previously
  df = pd.read_csv(direct + "/" + file_name + ".csv")
  df.pop("Year")

  # Recreate the same training data as before
  n = len(df)
  train_df = df[0:int(n*0.7)]

  # Normalize data once again
  train_mean = train_df.mean()
  train_std = train_df.std()

  df = (df - train_mean) / train_std

  # Create a valid dataset and retrieve predictions
  time_steps = 19
  X_full_data = create_dataset_X(df, time_steps)
  df_pred = new_model.predict(X_full_data)

  # Normalize and pull yields out of the .predict output
  normalized_pred = denormalize_pred(df_pred, train)
  yield_predictions = []
  for predictions in normalized_pred:
    yield_predictions.append(predictions[7])

  # Find the average of every 19 window steps that was used in model creation
  all_predictions = []
  for i in range(0, 627):
    current_predictions = yield_predictions[i*19: (i+1)*19]
    all_predictions.append(np.mean(current_predictions))

  # Add predictions to the CSV and save it to a new output file
  df = pd.read_csv(direct + "/" + f)
  df = df[19:]
  df["Predict"] = all_predictions

  #Create an output folder named [cropname]_predict, change directory here
  df.to_csv("/content/drive/MyDrive/GEOG481/output/{0}/Predict/{1}_PREDICT.csv".format(crop_type,file_name.split(".")[0]), index=False)

# Model Creation Iteration

This is the section used for iterating through all usable models and creating the models for each CARUID and crop type. Commented out so that new models aren't constantly recreated.

In [ ]:
# for file in usable_models:
#   print(file)
#   data_manipulation(file) # Manipulate Data
#   model_creation() # Create Models
  

# Model Training Iteration

This is the section used for iterating through all usable models and training the models for each CARUID and crop type. Commented out so that training isn't performed when it is not desired.

In [ ]:
# for file in usable_models: 
#     print(file) 
#     data_manipulation(file) # Manipulate Data
#     model_training() # Train Models

# Model Prediction Iteration

This is the section used for iterating through all usable models and grabbing the predicted values from their corresponding models for each CARUID and crop type. Commented out so that predictions aren't acquired when it is not desired.

In [ ]:
# for file in usable_models:
#   print(file)
#   grab_predictions(file) # Grab predictions